## Approached the problem word by word by taking out polarity with sentiment analyzer

In [1]:
import pandas as pd
import numpy as np


In [2]:
sentiment = pd.read_csv('C:/Users/Administrator/Desktop/feedback/sentiment.csv')
sentiment.shape

(12936, 4)

In [3]:
sentiment.head(10)

,RESPONSEID,Word 1,Word 2,Word 3
0,71517,Good place to start career,Proud to work with,NaN
1,71140,Friendly environment,Good support from Managers,Growth
2,69933,Employee friendly,Good Management,Supportive
3,75074,is the best,NaN,NaN
4,69559,Easy to complete task,Improve skill set,Appropriating employees talent
5,71600,Nice Environment to work,NaN,NaN
6,69954,Time Management,Discipline,Work environment
7,101421,Chandu,Chandu,Chandu
8,101422,1,2,3
9,69584,Shift Timings,Good appraisal Hike,NaN


In [4]:
#lets start working

#creating copy for any changes
sentiment_copy = sentiment.copy()

sentiment_copy = sentiment.drop('RESPONSEID', axis = 1)
sentiment_copy.shape

(12936, 3)

In [11]:
#importing required libraries for texts analysis
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

C:\Users\Administrator\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [26]:
# Some text cleaning for sentiment analysis.

docs_word_1 = sentiment_copy['Word 1'].fillna('').str.lower().str.replace('[^a-z ]','')
docs_word_2 = sentiment_copy['Word 2'].fillna('').str.lower().str.replace('[^a-z ]','')
docs_word_3 = sentiment_copy['Word 3'].fillna('').str.lower().str.replace('[^a-z ]','')

#removing stopwords
stopwords   = nltk.corpus.stopwords.words('english')
stemmer     = nltk.stem.PorterStemmer()

def clean_sentence(text):
    words = text.split(' ')
    words_clean = [stemmer.stem(word) for word in words if word not in stopwords]
    return ' '.join(words_clean)

docs_word_1 = docs_word_1.apply(clean_sentence)
docs_word_2 = docs_word_2.apply(clean_sentence)
docs_word_3 = docs_word_3.apply(clean_sentence)

doc = pd.concat([docs_word_1,docs_word_2,docs_word_3], axis = 1)
doc.head(5)

#now we have cleaned data , ready for sentiment analysis.

,Word 1,Word 2,Word 3
0,good place start career,proud work,
1,friendli environ,good support manag,growth
2,employe friendli,good manag,support
3,best,,
4,easi complet task,improv skill set,appropri employe talent


In [55]:
#creating a function for sentiment analysis

rating =[] #list to store rating for each word columnwise
def get_sentiment(text):
    sentiment = SentimentIntensityAnalyzer()
    compound  = sentiment.polarity_scores(text)['compound']
    rating.append(compound)
    
    if compound > 0.05:
        return 'Positive'
    elif compound < -0.05:
        return 'Negative'
    else:
        return 'Neutral'
#doc['Word 1 rating']    = doc['Word 1'].apply(get_sentiment)
#doc['Word 2 polarity']  = doc['Word 2'].apply(get_sentiment)
doc['Word 3 polarity']   = doc['Word 3'].apply(get_sentiment)


In [58]:
doc['word_3_rating'] =  rating
doc.columns

Index(['Word 1', 'Word 2', 'Word 3', 'Word 1 rating', 'word_1_rating',
       'Word 2 polarity', 'word_2_rating', 'Word 3 polarity', 'word_3_rating'],
      dtype='object')

In [71]:
#from scipy.stats import mode
doc['final polarity'] =  doc[['word_1_rating','word_3_rating','word_2_rating']].mean(axis = 1)
doc.head(10)
doc['final result']   = np.where(doc['final polarity'] >= 0.05,'positive','negative')
doc['final result']   = np.where(doc['final polarity'] <= -0.05,'negative',doc['final result'])
doc['final result']   = np.where((doc['final polarity'] > -0.05) & (doc['final polarity'] < 0.05) ,'neutral',doc['final result'])
doc.head(20)

,Word 1,Word 2,Word 3,Word 1 rating,word_1_rating,Word 2 polarity,word_2_rating,Word 3 polarity,word_3_rating,final polarity,final result
0,good place start career,proud work,,Positive,0.4404,Positive,0.4767,Neutral,0.0000,0.305700,positive
1,friendli environ,good support manag,growth,Neutral,0.0000,Positive,0.6808,Positive,0.3818,0.354200,positive
2,employe friendli,good manag,support,Neutral,0.0000,Positive,0.4404,Positive,0.4019,0.280767,positive
3,best,,,Positive,0.6369,Neutral,0.0000,Neutral,0.0000,0.212300,positive
4,easi complet task,improv skill set,appropri employe talent,Neutral,0.0000,Neutral,0.0000,Positive,0.4215,0.140500,positive
5,nice environ work,,,Positive,0.4215,Neutral,0.0000,Neutral,0.0000,0.140500,positive
6,time manag,disciplin,work environ,Neutral,0.0000,Neutral,0.0000,Neutral,0.0000,0.000000,neutral
7,chandu,chandu,chandu,Neutral,0.0000,Neutral,0.0000,Neutral,0.0000,0.000000,neutral
8,,,,Neutral,0.0000,Neutral,0.0000,Neutral,0.0000,0.000000,neutral
9,shift time,good apprais hike,,Neutral,0.0000,Positive,0.4404,Neutral,0.0000,0.146800,positive


In [73]:
doc.columns
doc = doc.rename(columns= {'Word 1 rating' : 'Word 1 polarity'}) 
doc.head(10)

,Word 1,Word 2,Word 3,Word 1 polarity,word_1_rating,Word 2 polarity,word_2_rating,Word 3 polarity,word_3_rating,final polarity,final result
0,good place start career,proud work,,Positive,0.4404,Positive,0.4767,Neutral,0.0000,0.305700,positive
1,friendli environ,good support manag,growth,Neutral,0.0000,Positive,0.6808,Positive,0.3818,0.354200,positive
2,employe friendli,good manag,support,Neutral,0.0000,Positive,0.4404,Positive,0.4019,0.280767,positive
3,best,,,Positive,0.6369,Neutral,0.0000,Neutral,0.0000,0.212300,positive
4,easi complet task,improv skill set,appropri employe talent,Neutral,0.0000,Neutral,0.0000,Positive,0.4215,0.140500,positive
5,nice environ work,,,Positive,0.4215,Neutral,0.0000,Neutral,0.0000,0.140500,positive
6,time manag,disciplin,work environ,Neutral,0.0000,Neutral,0.0000,Neutral,0.0000,0.000000,neutral
7,chandu,chandu,chandu,Neutral,0.0000,Neutral,0.0000,Neutral,0.0000,0.000000,neutral
8,,,,Neutral,0.0000,Neutral,0.0000,Neutral,0.0000,0.000000,neutral
9,shift time,good apprais hike,,Neutral,0.0000,Positive,0.4404,Neutral,0.0000,0.146800,positive


# THANK YOU ~~